# LangGraph Agent with @agent_config

A simple agent with nodes A, B, C where routing is controlled by config.

## Define the Agent

In [ ]:
from dataclasses import dataclass, field
from typing import TypedDict

from langgraph.graph import StateGraph, END
from opik.integrations.langchain import track_langgraph, OpikTracer

from opik_config import Prompt, agent_config


@agent_config
@dataclass
class AgentConfig:
    use_path_b: bool = True
    prompt_a: Prompt = field(default_factory=lambda: Prompt(name="SystemPromptA", prompt="You are node A. Analyze the input."))
    prompt_b: Prompt = field(default_factory=lambda: Prompt(name="AIMessageB", prompt="You are node B. Summarize concisely."))
    model: str = "gpt-4"


config = AgentConfig()




class State(TypedDict):
    input: str
    output: str


def node_a(state: State) -> State:
    prompt = config.prompt_a
    print(f"[Node A] Using prompt: {prompt}")
    return {"input": state["input"], "output": f"A processed: {state['input']}"}


def node_b(state: State) -> State:
    prompt = config.prompt_b
    print(f"[Node B] Using prompt: {prompt}")
    return {"input": state["input"], "output": f"B summarized: {state['output']}"}


def node_c(state: State) -> State:
    print(f"CCCCC")
    print(config.model)
    return {"input": state["input"], "output": f"C expanded: {state['output']}"}


def route_from_a(state: State) -> str:
    if config.use_path_b:
        print("[Router] Taking path B")
        return "node_b"
    else:
        print("[Router] Taking path C")
        return "node_c"

# Build the graph
graph = StateGraph(State)
graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)

graph.set_entry_point("node_a")
graph.add_conditional_edges("node_a", route_from_a, {"node_b": "node_b", "node_c": "node_c"})
graph.add_edge("node_b", END)
graph.add_edge("node_c", END)

tracer = OpikTracer(project_name="config-demo")
agent = track_langgraph(graph.compile(), tracer)

## Run with default config (path B)

In [2]:
def run_agent(user_input: str) -> str:
    result = agent.invoke({"input": user_input, "output": ""})
    return result["output"]


print("=== Default config (use_path_b=True) ===")
output = run_agent("Hello world")
print(f"Result: {output}")

OPIK: Started logging traces to the "Default Project" project at http://localhost:5174/api/v1/session/redirect/projects/?trace_id=019c0b62-47fd-7b8b-86f2-10897181db16&path=aHR0cDovL2xvY2FsaG9zdDo1MTc0L2FwaS8=.


=== Default config (use_path_b=True) ===
[Node A] Using prompt: You are node A. Analyze the input.
[Router] Taking path B
[Node B] Using prompt: You are node B. Summarize concisely.
Result: B summarized: A processed: Hello world


In [4]:
from opik_config import experiment_context

EXPERIMENT_ID = "8d826426-8e81-444d-b3df-ae0dfebb064d"

experiment_id = str(EXPERIMENT_ID)
with experiment_context(experiment_id):
    output = run_agent("Hello world")
    print(f"Result: {output}")


[Node A] Using prompt: You are node A. Analyze the input.
[Router] Taking path C
CCCCC
gpt-4
Result: C expanded: A processed: Hello world


## Set up experiment overrides

Start the config backend first:
```bash
uv run python -m opik_config
```

In [ ]:
import requests
import uuid

CONFIG_URL = "http://localhost:5050"

experiment_id = str(uuid.uuid4())
print(f"Experiment ID: {experiment_id}")

# Override: take path C instead of B
requests.post(f"{CONFIG_URL}/config/set", json={
    "key": "use_path_b",
    "value": False,
    "experiment_id": experiment_id
})

# Override: change prompt_c
requests.post(f"{CONFIG_URL}/config/set", json={
    "key": "prompt_c",
    "value": "You are an EXPERIMENTAL node C. Be very detailed and thorough.",
    "experiment_id": experiment_id
})

print("Overrides set: use_path_b=False, prompt_c updated")

## Run with experiment context (path C)

In [ ]:
from opik_config import experiment_context

print("=== With experiment (use_path_b=False, new prompt_c) ===")
with experiment_context(experiment_id):
    output = run_agent("Hello world")
print(f"Result: {output}")

## Run again without experiment (back to path B)

In [ ]:
print("=== Without experiment context (defaults) ===")
output = run_agent("Hello again")
print(f"Result: {output}")